In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sbn
import sklearn as skl
from ucimlrepo import fetch_ucirepo

In [4]:
automobile = fetch_ucirepo(id=10)
data = automobile.data.original
print(data)

     normalized-losses         make fuel-type aspiration  num-of-doors  \
0                  NaN  alfa-romero       gas        std           2.0   
1                  NaN  alfa-romero       gas        std           2.0   
2                  NaN  alfa-romero       gas        std           2.0   
3                164.0         audi       gas        std           4.0   
4                164.0         audi       gas        std           4.0   
..                 ...          ...       ...        ...           ...   
200               95.0        volvo       gas        std           4.0   
201               95.0        volvo       gas      turbo           4.0   
202               95.0        volvo       gas        std           4.0   
203               95.0        volvo    diesel      turbo           4.0   
204               95.0        volvo       gas      turbo           4.0   

      body-style drive-wheels engine-location  wheel-base  length  ...  \
0    convertible          rwd        